## Lab 5 Hash and Tries

### Exercise 1
Modify the rehash operation such that each time a new random hash function is used. Use universal hashing.

Here we will implement a universal-1 hashing as an example.

In [119]:
import numpy as np
import math
class HashSet:
    
    ## Define you new hash function here
    def __hash(self, elem):
        sum_product = 0
        if (type(elem) == int):
            return (elem * self.a[0]) % self.p
        for i in range(self.k):
            sum_product += elem[i] * self.a[i]
        return sum_product % self.p
        
    def __init__(self,contents=[]):
        self.items = [None] * 17
        self.p = 17 # prime number P
        if (type(contents[0]) == tuple):
            self.k = len(contents[0])
        else:
            self.k = 1
        # add w & a here
        self.w = math.floor(math.log(self.p, 2))
        self.a = tuple(np.random.randint(0,self.p, self.k))
        self.numItems = 0
        for e in contents:
            self.add(e)
            
    def add(self,item):
        if HashSet.__add(self, item,self.items):
            self.numItems += 1
            load = self.numItems / len(self.items)
            if load >= 0.75:
                self.items = HashSet.__rehash(self, self.items, [None]*2*len(self.items))
                
    def __contains__(self,item):
#         index = hash(item) % len(self.items)
        index = self.__hash(item)
        while self.items[index] != None:
            if self.items[index] == item:
                return True
            index = (index + 1) % len(self.items)
        return False
    
    def delete(self,item):
        if HashSet.__remove(self,item,self.items):
            self.numItems -= 1
            load = max(self.numItems,20) / len(self.items)
            if load <= 0.25:
                self.items = HashSet.__rehash(self, self.items, [None]*(len(self.items) // 2))
        else:
            raise KeyError("Item not in HashSet")

    # ===== Hidden Class =====
    class __Placeholder:
        def __init__(self):
            pass
        
        def __eq__(self,other):
            return False
    
    # ===== Auxiliary Functions =====
    # They all have '__' as prefixes to indicate that they are private methods to the class
    def __add(self, item,items):
        index = self.__hash(item) # hash() is used to produce a number for item
        location = -1
        pt_test = 0
        while items[index] != None:
            if items[index] == item:
                return False
            if location < 0 and type(items[index]) == HashSet.__Placeholder:
                location = index
            index = (index + 1) % len(items)
            pt_test = 1
        if(pt_test == 1):
             print("collision!")
        if location < 0:
            location = index
        items[location] = item
        return True
        
    def __rehash(self, olditems,newitems):
    ## Modify here
        # update p, w, and a
        self.p = len(newitems) # A relaxed method
        self.w = math.floor(math.log(self.p, 2))
        self.a = tuple(np.random.randint(0,self.p,self.k))
        self.items = newitems
        for e in olditems:
            if type(e) == int or type(e) == tuple:
                HashSet.__add(self, e,newitems)
        return newitems
                
    def __remove(self, item,items):
        index = self.__hash(item)
        while items[index] != None:
            if items[index] == item:
                nextIndex = (index + 1) % len(items)
                if items[nextIndex] == None:
                    items[index] = None
                else:
                    items[index] = HashSet.__Placeholder()
                return True
            index = (index + 1) % len(items)
        return False




In [120]:
# try some basic test here
hs = HashSet([(i, i+1) for i in range(17)])
print(hs.items)
# Add
hs.add((40, 41))
print(hs.items)
# Delete
hs.delete((1, 2))
print(hs.items)
# Add
hs.add((1, 2))
for i in range(55, 70):
    hs.add((i, i+1))
print(hs.items)
print(len(hs.items))
print(hs.p)
print(hs.w)
print(hs.a)

[None, None, (12, 13), None, (4, 5), None, None, (9, 10), None, (1, 2), (14, 15), None, (6, 7), None, None, (11, 12), None, (3, 4), (16, 17), None, (8, 9), None, (0, 1), (13, 14), None, (5, 6), None, None, (10, 11), None, (2, 3), (15, 16), None, (7, 8)]
collision!
[None, None, (12, 13), None, (4, 5), None, None, (9, 10), None, (1, 2), (14, 15), None, (6, 7), (40, 41), None, (11, 12), None, (3, 4), (16, 17), None, (8, 9), None, (0, 1), (13, 14), None, (5, 6), None, None, (10, 11), None, (2, 3), (15, 16), None, (7, 8)]
[None, None, (12, 13), None, (4, 5), None, None, (9, 10), None, <__main__.HashSet.__Placeholder object at 0x11f535c50>, (14, 15), None, (6, 7), (40, 41), None, (11, 12), None, (3, 4), (16, 17), None, (8, 9), None, (0, 1), (13, 14), None, (5, 6), None, None, (10, 11), None, (2, 3), (15, 16), None, (7, 8)]
collision!
collision!
collision!
[None, None, None, (15, 16), (6, 7), (65, 66), (56, 57), None, None, None, None, (11, 12), (2, 3), (61, 62), None, None, None, None, (16, 

In [121]:
# collision test
hs = HashSet([(2, 4)])
print(hs.items)
print(hs.w)
print(hs.a)

[None, None, None, None, None, None, None, None, None, None, (2, 4), None, None, None, None, None, None]
4
(15, 12)


### Exercise 2 (Linear hashing)
Implement a simple variant of linear hashing as an alternative to hashing with chaining. Fix a maximum length l of the lists stored in a table entry.

Fix a number m and a hash function h with values in {0,...,m−1}. The hash table will have a size M ≥ m. Then define inductively hash functions hi with h0 = h and hj+1(e) = hj(e) % m · 2j, so hj takes 2j · m different hash values.
The hash value of an element e is hj(e), where j is maximal with hj(e) ≤ M − 1.

(i) Show that whenever an element e is given, it suffices to compute at most two
hash values.

(ii) Implement a rehash operation that comes into action, when the maximum list length l will be exceeded by insertion of a new element e. In this case use hj+1 to distribute the elements of the list into two sublists and increase M accordingly. If necessary, split other lists as well according to the new value M.

(iii) Show that if it is permitted to use also hj−1 instead of hj to determine the hash value of an element e, then rehashing can be done in a lazy way by splitting only one list.

In [122]:
## modify the hash set to linear hashing
class HashSet:
    def __init__(self, m = 4, l = 4, contents=[]):
        self.mod_val = m # set it to m
        self.m = m # constant
        self.M = m # first set it the same size as m
        self.j = 1 # start one is j_0
        self.l = l # the miximum length for each hash table
        self.items = {} # it is a dictionary, coresspond to each table
        self.content = [] # record all the elements up to now
#         self.numItems = 0
        for i in range(self.M):
            self.items[str(i)] = []
        for i in range(len(contents)):
            self.add(contents[i])
            
    def add(self,item):
        index = item % self.mod_val
        if(len(self.items[str(index)]) == self.l):
            self.items = self.__rehash()
            index = item % self.mod_val  # do the hash according to hj
        self.items[str(index)].append(item)
        self.content.append(item)
        return
                
    def __contains__(self,item):
        index = item % self.mod_val
        if item in self.items[index]:
            return True
        return False
    
    def delete(self,item):
        index = item % self.mod_val
        if(len(self.items[str(index)]) == 0):
            raise Error("Hash table is already empty")
        if(item in self.items[str(index)]):
            self.content.remove(item)
            self.items[str(index)].remove(item)
#             self.numItems -= 1
#             load = max(self.numItems,20) / len(self.items)
#             if load <= 0.25:
#                 self.items = HashSet.__rehash(self.items, [None]*(len(self.items) // 2))
        else:
            raise KeyError("Item not in HashSet")

#     # ===== Hidden Class =====
#     class __Placeholder:
#         def __init__(self):
#             pass
        
#         def __eq__(self,other):
#             return False
    
    # ===== Auxiliary Functions =====
    # They all have '__' as prefixes to indicate that they are private methods to the class
#     def __add(self, item):
#         index = item % self.m  # see which hash table to go to
# #         location = -1
# #         while items[index] != None:
# #             if items[index] == item:
# #                 return False
# #             if location < 0 and type(items[index]) == HashSet.__Placeholder:
# #                 location = index
# #             index = (index + 1) % len(items)
# #         if location < 0:
# #             location = index
# #         items[location] = item
# #         return True
        
        
    def __rehash(self):
        self.mod_val  = self.m * 2**self.j
        self.j += 1
        newItem = {}
        max_one = float('-inf')
        for elem in self.content:
            index = elem % self.mod_val
            if(index > max_one):
                max_one = index
        self.M = max_one + 1
        for i in range(self.M):
            newItem[str(i)] = []
        for elem in self.content:
            index = elem % self.mod_val
            newItem[str(index)].append(elem)
#             self.numItems += 1
        
#         for e in olditems:
#             if e != None and type(e) != HashSet.__Placeholder:
#                 HashSet.__add(e,newitems)
        return newItem
                
#     def __remove(item,items):
#         index = hash(item) % len(items)
#         while items[index] != None:
#             if items[index] == item:
#                 nextIndex = (index + 1) % len(items)
#                 if items[nextIndex] == None:
#                     items[index] = None
#                 else:
#                     items[index] = HashSet.__Placeholder()
#                 return True
#             index = (index + 1) % len(items)
#         return False

In [124]:
s = HashSet(4,4,[i+1 for i in range(16)])
print(s.items)
s.add(20)
print(s.items)
s.delete(2)
s.delete(4)
print(s.items)

{'0': [4, 8, 12, 16], '1': [1, 5, 9, 13], '2': [2, 6, 10, 14], '3': [3, 7, 11, 15]}
{'0': [8, 16], '1': [1, 9], '2': [2, 10], '3': [3, 11], '4': [4, 12, 20], '5': [5, 13], '6': [6, 14], '7': [7, 15]}
{'0': [8, 16], '1': [1, 9], '2': [10], '3': [3, 11], '4': [12, 20], '5': [5, 13], '6': [6, 14], '7': [7, 15]}


### Exercise 3
(i) Modify the implementation of the class Trie used in the lectures such that it
suffices to store unique prefixes of minimum length of the keys in a Trie object.

(ii) Use this idea to store the elements of a set S that is in one-one correspondence
with the set K of keys in a Trie object.

(iii) Modify the insert and find operations accordingly.

In [253]:
## modify the class to store shortest unique prefix and modify insert & contain operations
class Trie:
    class TrieNode:
        def __init__(self,item,next=None,follows=None): 
            self.item = item
            self.next = next
            self.pre = None
            self.follows = follows 
    def __init__(self):
        self.start = None 
        self.key_endnode_dict = {}
        self.prefix_dict = {}
    def insert(self,item):
        keys = ''.join(item)
        self.start = Trie.__insert(self, self.start,item, keys)
#         for key in self.key_endnode_dict.keys():
#             node = self.key_endnode_dict[key]
#             suffix = ''
#             if(node.next != None):
#                 suffix = ''
#             else:
#                 while(node.pre != None and node.pre.next == None):
#                     suffix += node.item
#                     node = node.pre
#                 suffix = list(suffix[:-1])
#                 suffix.reverse()
#             prefix = list(keys)[:(len(item_c) - len(suffix))]
#             self.prefix_dict[keys] = prefix
        self.findPrefix()
            
    def findPrefix(self):
        for key in self.key_endnode_dict.keys():
            node = self.key_endnode_dict[key]
            suffix = ''
            if(node.next != None):
                suffix = ''
            else:
                while(node.pre != None and node.next == None):
                    suffix += node.item
                    node = node.pre
                suffix = list(suffix[:-1])
                suffix.reverse()
            prefix = list(key)[:(len(key) - len(suffix))]
            self.prefix_dict[key] = prefix
            
                
#         self.key_endnode_dict[item] = # last node
    def __insert(self, node,item, keys): 
        if item == []:
            newnode = Trie.TrieNode("#")
            self.key_endnode_dict[keys] = newnode
            return newnode
        if node == None:
            key = item.pop(0)
#             print(key)
            newnode = Trie.TrieNode(key)
            newnode.follows = Trie.__insert(self, newnode.follows,item, keys) 
            newnode.follows.pre = newnode
            return newnode
        else:
            key = item[0]
#             print(key)
            if node.item == key:
                key = item.pop(0)
                node.follows = Trie.__insert(self, node.follows,item, keys)
                node.follows.pre = node
            else:
                node.next = Trie.__insert(self, node.next,item,keys)
#                 node.next.pre = node
            return node
    # a print function which can print out structure of tries 
    # to help better understand
    def print_trie(self, root, level_f):
        if(root == None):
            return
        if(root.item != '#'):
            print(root.item, '-', end='')
        else:
            print(root.item, end='')  
        self.print_trie(root.follows, level_f+1)
        if(root.next!=None):
            print('\n')
            str_sp = ' '*level_f*3 
            print(str_sp+'|')
            print(str_sp, end='')
        self.print_trie(root.next,level_f)
        return
    
    def __contains__(self,item):
        if(''.join(item) in self.prefix_dict.keys()):
            return True
        return False
            

In [256]:
t1 = Trie()
t1.insert(['c', 'o', 'w','s'])
t1.insert(['c', 'a', 't'])
t1.insert(['r', 'a', 't'])
# t1.print_trie(t1.start)
t1.insert(['r', 'a', 'b', 'b', 'i', 't'])
t1.insert(['r', 'a', 'b', 'b', 'i', 't','s'])
t1.insert(['r', 'a', 'b', 'b','i','t','s','u','i','t'])
t1.insert(['r', 'a', 'b', 'b','i','t','s','h','i','t'])
t1.insert(['r', 'a', 'i','n'])
# t1.print_trie(t1.start)
t1.insert(['d', 'o', 'g'])
t1.insert(['c', 'o', 'w', 'w', 'w'])
t1.print_trie(t1.start, 0)
print('\n',t1.prefix_dict)

['c', 'a', 'b'] in t1


c -o -w -s -#

         |
         w -w -#

   |
   a -t -#

|
r -a -t -#

      |
      b -b -i -t -#

                  |
                  s -#

                     |
                     u -i -t -#

                     |
                     h -i -t -#

      |
      i -n -#

|
d -o -g -#
 {'cows': ['c', 'o', 'w', 's'], 'cat': ['c', 'a'], 'rat': ['r', 'a', 't'], 'rabbit': ['r', 'a', 'b', 'b', 'i', 't'], 'rabbits': ['r', 'a', 'b', 'b', 'i', 't', 's'], 'rabbitsuit': ['r', 'a', 'b', 'b', 'i', 't', 's', 'u'], 'rabbitshit': ['r', 'a', 'b', 'b', 'i', 't', 's', 'h'], 'rain': ['r', 'a', 'i'], 'dog': ['d'], 'cowww': ['c', 'o', 'w', 'w']}


False